# Load data

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
# Load and split data
filename = '../data/bankruptcy.csv'
df = pd.read_csv(filename, sep=',', index_col=False)

# Drop column (always same value)
df.drop(columns=[' Net Income Flag'], inplace=True)

# Drop two outlier rows (encoding errors)
df.drop(df[df[' Revenue per person'] > 1].index, inplace=True)

# Split into X, Y
values = df.values
X, Y = values[:, 1:], values[:, 0]
feature_names = list(df.columns)[1:]

# Train/val/test split
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=0)

In [3]:
# Neural network pre-processing
enc = OneHotEncoder(sparse=False)
Y_train_oh = enc.fit_transform(np.expand_dims(Y_train, -1))
Y_val_oh = enc.fit_transform(np.expand_dims(Y_val, -1))
Y_test_oh = enc.transform(np.expand_dims(Y_test, -1))

ss = StandardScaler()
ss.fit(X_train)
X_train_std = ss.transform(X_train)
X_val_std = ss.transform(X_val)
X_test_std = ss.transform(X_test)

# Set up imputer

In [4]:
import torch
import fastshap_torch

In [5]:
device = torch.device('cuda', 1)
surrogate = torch.load('../models/bankruptcy_surrogate.pt').eval().to(device)

In [6]:
def imputer(x, S):
    x = torch.tensor(x, dtype=torch.float32, device=device)
    S = torch.tensor(S, dtype=torch.float32, device=device)
    pred = surrogate((x, S)).softmax(dim=-1)
    return pred.cpu().data.numpy()

# Ground truth SHAP values

In [7]:
import pickle
import matplotlib.pyplot as plt
from shapreg import games, shapley, shapley_sampling

In [8]:
num_examples = 256
thresh = 0.005
# thresh = 0.01

In [9]:
shap_values = []

for i in range(num_examples):
    # Get instance
    x = X_test_std[i]

    # Set up game
    game = games.PredictionGame(imputer, x)
    
    # Calculate ground truth SHAP values
    explanation = shapley.ShapleyRegression(game, thresh=thresh, bar=False)
    shap_values.append(explanation.values.T)
    print('Done with sample = {}'.format(i))

Done with sample = 0
Done with sample = 1
Done with sample = 2
Done with sample = 3
Done with sample = 4
Done with sample = 5
Done with sample = 6
Done with sample = 7
Done with sample = 8
Done with sample = 9
Done with sample = 10
Done with sample = 11
Done with sample = 12
Done with sample = 13
Done with sample = 14
Done with sample = 15
Done with sample = 16
Done with sample = 17
Done with sample = 18
Done with sample = 19
Done with sample = 20
Done with sample = 21
Done with sample = 22
Done with sample = 23
Done with sample = 24
Done with sample = 25
Done with sample = 26
Done with sample = 27
Done with sample = 28
Done with sample = 29
Done with sample = 30
Done with sample = 31
Done with sample = 32
Done with sample = 33
Done with sample = 34
Done with sample = 35
Done with sample = 36
Done with sample = 37
Done with sample = 38
Done with sample = 39
Done with sample = 40
Done with sample = 41
Done with sample = 42
Done with sample = 43
Done with sample = 44
Done with sample = 4

In [10]:
with open('../results/bankruptcy_shap.pkl', 'wb') as f:
    pickle.dump(shap_values, f)

# Estimation curves

In [36]:
samples = 8192
num_features = X_train.shape[1]

In [46]:
kernelshap_curves = []

for i in range(num_examples):
    # Get instance
    x = X_test_std[i]

    # Set up game
    game = games.PredictionGame(imputer, x)
    
    # Calculate ground truth SHAP values
    results = shapley.ShapleyRegression(game, batch_size=64, n_samples=8 * samples, detect_convergence=False,
                                        bar=False, paired_sampling=False, return_all=True)
    curve = np.array([explanation.T for explanation in results[1]['values']])
    kernelshap_curves.append(curve)
    print('Done with sample = {}'.format(i))

kernelshap_iters = results[1]['iters']

Done with sample = 0
Done with sample = 1
Done with sample = 2
Done with sample = 3
Done with sample = 4
Done with sample = 5
Done with sample = 6
Done with sample = 7
Done with sample = 8
Done with sample = 9
Done with sample = 10
Done with sample = 11
Done with sample = 12
Done with sample = 13
Done with sample = 14
Done with sample = 15
Done with sample = 16
Done with sample = 17
Done with sample = 18
Done with sample = 19
Done with sample = 20
Done with sample = 21
Done with sample = 22
Done with sample = 23
Done with sample = 24
Done with sample = 25
Done with sample = 26
Done with sample = 27
Done with sample = 28
Done with sample = 29
Done with sample = 30
Done with sample = 31
Done with sample = 32
Done with sample = 33
Done with sample = 34
Done with sample = 35
Done with sample = 36
Done with sample = 37
Done with sample = 38
Done with sample = 39
Done with sample = 40
Done with sample = 41
Done with sample = 42
Done with sample = 43
Done with sample = 44
Done with sample = 4

In [44]:
paired_curves = []

for i in range(num_examples):
    # Get instance
    x = X_test_std[i]

    # Set up game
    game = games.PredictionGame(imputer, x)
    
    # Calculate ground truth SHAP values
    results = shapley.ShapleyRegression(game, batch_size=64, n_samples=2 * (samples / 2), detect_convergence=False,
                                        bar=False, paired_sampling=True, return_all=True)
    curve = np.array([explanation.T for explanation in results[1]['values']])
    paired_curves.append(curve)
    print('Done with sample = {}'.format(i))

paired_iters = results[1]['iters']

Done with sample = 0
Done with sample = 1
Done with sample = 2
Done with sample = 3
Done with sample = 4
Done with sample = 5
Done with sample = 6
Done with sample = 7
Done with sample = 8
Done with sample = 9
Done with sample = 10
Done with sample = 11
Done with sample = 12
Done with sample = 13
Done with sample = 14
Done with sample = 15
Done with sample = 16
Done with sample = 17
Done with sample = 18
Done with sample = 19
Done with sample = 20
Done with sample = 21
Done with sample = 22
Done with sample = 23
Done with sample = 24
Done with sample = 25
Done with sample = 26
Done with sample = 27
Done with sample = 28
Done with sample = 29
Done with sample = 30
Done with sample = 31
Done with sample = 32
Done with sample = 33
Done with sample = 34
Done with sample = 35
Done with sample = 36
Done with sample = 37
Done with sample = 38
Done with sample = 39
Done with sample = 40
Done with sample = 41
Done with sample = 42
Done with sample = 43
Done with sample = 44
Done with sample = 4

In [48]:
sampling_curves = []

for i in range(num_examples):
    # Get instance
    x = X_test_std[i]

    # Set up game
    game = games.PredictionGame(imputer, x)
    
    # Calculate ground truth SHAP values
    results = shapley_sampling.ShapleySampling(game, batch_size=1, n_samples=int(np.ceil(samples / num_features)), detect_convergence=False,
                                               bar=False, return_all=True)
    curve = np.array([explanation for explanation in results[1]['values']])
    sampling_curves.append(curve)
    print('Done with sample = {}'.format(i))

sampling_iters = results[1]['iters']

Done with sample = 0
Done with sample = 1
Done with sample = 2
Done with sample = 3
Done with sample = 4
Done with sample = 5
Done with sample = 6
Done with sample = 7
Done with sample = 8
Done with sample = 9
Done with sample = 10
Done with sample = 11
Done with sample = 12
Done with sample = 13
Done with sample = 14
Done with sample = 15
Done with sample = 16
Done with sample = 17
Done with sample = 18
Done with sample = 19
Done with sample = 20
Done with sample = 21
Done with sample = 22
Done with sample = 23
Done with sample = 24
Done with sample = 25
Done with sample = 26
Done with sample = 27
Done with sample = 28
Done with sample = 29
Done with sample = 30
Done with sample = 31
Done with sample = 32
Done with sample = 33
Done with sample = 34
Done with sample = 35
Done with sample = 36
Done with sample = 37
Done with sample = 38
Done with sample = 39
Done with sample = 40
Done with sample = 41
Done with sample = 42
Done with sample = 43
Done with sample = 44
Done with sample = 4

In [49]:
antithetical_curves = []

for i in range(num_examples):
    # Get instance
    x = X_test_std[i]

    # Set up game
    game = games.PredictionGame(imputer, x)
    
    # Calculate ground truth SHAP values
    results = shapley_sampling.ShapleySampling(game, batch_size=2, n_samples=int(np.ceil(samples / num_features)), detect_convergence=False,
                                               bar=False, antithetical=True, return_all=True)
    curve = np.array([explanation for explanation in results[1]['values']])
    antithetical_curves.append(curve)
    print('Done with sample = {}'.format(i))

antithetical_iters = results[1]['iters']

Done with sample = 0
Done with sample = 1
Done with sample = 2
Done with sample = 3
Done with sample = 4
Done with sample = 5
Done with sample = 6
Done with sample = 7
Done with sample = 8
Done with sample = 9
Done with sample = 10
Done with sample = 11
Done with sample = 12
Done with sample = 13
Done with sample = 14
Done with sample = 15
Done with sample = 16
Done with sample = 17
Done with sample = 18
Done with sample = 19
Done with sample = 20
Done with sample = 21
Done with sample = 22
Done with sample = 23
Done with sample = 24
Done with sample = 25
Done with sample = 26
Done with sample = 27
Done with sample = 28
Done with sample = 29
Done with sample = 30
Done with sample = 31
Done with sample = 32
Done with sample = 33
Done with sample = 34
Done with sample = 35
Done with sample = 36
Done with sample = 37
Done with sample = 38
Done with sample = 39
Done with sample = 40
Done with sample = 41
Done with sample = 42
Done with sample = 43
Done with sample = 44
Done with sample = 4

In [50]:
with open('../results/bankruptcy_curves.pkl', 'wb') as f:
    save_dict = {
        'kernelshap': kernelshap_curves,
        'kernelshap_iters': kernelshap_iters,

        'paired_sampling': paired_curves,
        'paired_sampling_iters': paired_iters,

        'sampling_curves': sampling_curves,
        'sampling_iters': sampling_iters,
        
        'antithetical_curves': antithetical_curves,
        'antithetical_iters': antithetical_iters,
    }
    pickle.dump(save_dict, f)